<a href="https://colab.research.google.com/github/maychr/NLP_AES_promt-based/blob/main/Prompt_essay_classifications_with_transformer_models/GPT_J_6B_few_shot_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation of the Environment
## Colab

In [ ]:
# Check GPU and print info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


Mon Mar 14 11:40:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Installation of the following additional packages
!pip install -q git+https://github.com/huggingface/transformers.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.5 MB 7.6 MB/s 
     |████████████████████████████████| 895 kB 84.9 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 75.1 MB/s 


# data preparation

In [ ]:
# Import test dataset from GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir("//content/gdrive/MyDrive/nlp_data/") 

import numpy
import pandas as pd
data = pd.read_csv("test_recoded_final.csv", sep= ";", encoding="UTF-8")


In [ ]:
# Casting the data into lists
label = data.iloc[0:, 1].to_list()
mail = data.iloc[0:, 0].to_list()


# Few-shot
## Download and Preparation

In [ ]:
# Download the transformer-based large language model (GPT-J6B) and tokenizer
import torch
from transformers import GPTJForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# We load the half-precision floating-point formated model to match the resources
model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", low_cpu_mem_usage=True)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")


Downloading:   0%|          | 0.00/836 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [ ]:
# Test run
# This funny statement checks whether the model works properly. We think this statement is crucial, otherwise "they" might refuse to work with you. ;-)
prompt = "I, For One, Welcome Our AI Overlords "
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_ids = model.generate(input_ids, do_sample=True, temperature=1, top_p=0.0, max_new_tokens=15)
generated_text = tokenizer.decode(generated_ids[0])
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I, For One, Welcome Our AI Overlords 

I, For One, Welcome Our AI Overlords 




In [ ]:
# Counter for prediction loop
count = 0


0

## Run the Model

In [ ]:
lst = ""

# Definition of few-shot prompt examples
few_shot = ("""Klassifizier die Mail. Labels: höflich, unhöflich
  ####
  mail: Hallo Susanne, unter gegebenen Umständen kann ich die folgenden Aufgaben nicht bearbeiten. Die Excel-Simulation ist nicht dafür geeignet. Darüber hinaus funktioniert mein Taschenrechner nicht. Mit freundlichen Grüßen
  Formulierung ist höflich
  #### 
  mail: bitte kommen Sie rüber und erklären mir es selbst
  Formulierung ist unhöflich
  ####
  mail: Guden, Ich habe keinerlei Ahnung ;) Gruß der Boss! 
  Formulierung ist unhöflich
  ####
  mail: Flexi
  Formulierung ist unhöflich
  ####
  mail: Sehr geehrte Frau Meier, ich habe die von Ihnen gestellte Aufgabe leider nicht lösen können, da ich im Umgang mit solchen Aufgaben noch nicht so viel Erfahrung habe. 
  Formulierung ist höflich
  ####
  """)

runs = len(mail)

# Run data in a loop
for count in range(len(mail)+1):
  if count == runs: 
    break

# Define the prompt with examples and mails to classify
  prompt = (few_shot,'\n',"mail: ",mail[count],'\n',"Formulierung ist")
    
  prompt = ''.join(prompt)

  count += 1

  texts = prompt

# Define Tokenization and Generation
  tokenizer.padding_side = "left"
  tokenizer.pad_token = tokenizer.eos_token
  encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
  with torch.no_grad():
      generated_ids = model.generate(**encoding, do_sample=True, temperature=.1, top_p=.9, max_new_tokens=4) #control model output
  generated_texts = tokenizer.batch_decode(
      generated_ids, skip_special_tokens=True)

# Print output for observing the classification during the prediction process (but consumes resources)
  for text in generated_texts:
    print("####")
    output = text.replace(few_shot, "")
    print(output, '\n', count-1, '\n')
    
    lst += output

print(count, 'Out of loop') 

torch.cuda.empty_cache()

## Further Processing of Model Output

In [ ]:
# Split mails
def Splithashtags(x):
  li = x.split("mail") 

  return li

ai_MailAndLabel = Splithashtags(lst)


In [ ]:
# Split labels
labels = []

def Splitlabels(x):

  for i in x:
    x = i.split('Formulierung ist ')
    if len(x) > 1:
        x = x[1].replace('\n', '').replace("'", '').replace(': ','').replace('####', '')
        labels.append(x)
  return labels

pred_class = Splitlabels(ai_MailAndLabel)
len(pred_class)


627

In [ ]:
# Check for undesired predicted labels
element = ["höflich","unhöflich"]

for x in pred_class:
    if x not in element:
        print(x)


In [ ]:
# Replace labels with 0 and 1 for evaulation
import pandas as pd
test_pred_class = pd.DataFrame(data=pred_class)

test_pred_class = test_pred_class.replace({'höflich':1, 'unhöflich':0})


In [ ]:
# Get human labels for evaluation
test_label = data.iloc[:,1]


# Calculate Evaluation Metrics

In [ ]:
# Checking the test data results by comparing human gold-labels and model prediction

from sklearn import metrics

test_label = test_label[0:runs]
test_pred_class = test_pred_class

# Mean accuracy
print("Mean Accuracy:\n", metrics.accuracy_score(test_label, test_pred_class))

# Confusion matrix
print("Confusion Matrix:\n", metrics.confusion_matrix(test_label, test_pred_class))

# F1 Score
print("F1 Score:\n", metrics.f1_score(test_label, test_pred_class))

# Precision
print("Precision:\n", metrics.precision_score(test_label, test_pred_class))

# Recall
print("Recall:\n", metrics.recall_score(test_label, test_pred_class))

# ROC AUC Score
print("ROC AUC:\n", metrics.roc_auc_score(test_label, test_pred_class))

# Cohen's Kappa Score
print("Cohen's Kappa:\n", metrics.cohen_kappa_score(test_label, test_pred_class))


Mean Accuracy:
 0.9298245614035088
Confusion Matrix:
 [[ 17  33]
 [ 11 566]]
F1 Score:
 0.9625850340136055
Precision:
 0.9449081803005008
Recall:
 0.9809358752166378
ROC AUC:
 0.6604679376083189
Cohen's Kappa:
 0.401639699822149


## Print Incorrect Predictions

We can further print incorrect predictions according to human gold-labels to

1.   Improve our few-shot prompt
2.   Check for human errors


In [ ]:
# Get incorrect classifications 
incorrects = test_label != test_pred_class[0]
sum(incorrects)


44

In [ ]:
# Print incorrect classifications
for input, prediction, label in zip(test_text, test_pred_class[0], test_label):
  if prediction != label:
    print(input, '\n\n', 'has been machine coded as', prediction, 'and should be', label, 'according to human coding.\n\n-------------------------------------------------------------------------------------\n')
